In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical

PNES_path ="../pilot/PNES/" # where the folders are saved
ES_path ="../pilot/ES/"

length = float('inf')
PNESdata = []
ESdata = []

In [2]:
pose_path = os.path.join(PNES_path, "OpenPose Pose")

for csv in os.listdir(pose_path):
    if csv.endswith(".csv") or csv.endswith(".CSV"):
        dataframe = pd.read_csv(os.path.join(pose_path, csv), header=None, skiprows=1)
        rolavg = dataframe.rolling(100).mean()
        rolavg = rolavg.dropna()
        rolavg = rolavg.reset_index(drop=True)
        PNESdata.append(rolavg.values)
        if rolavg.values.shape[0] < length:
            length = rolavg.values.shape[0]
            print(length)

3341
3301


In [3]:
pose_path = os.path.join(ES_path, "OpenPose Pose")

for csv in os.listdir(pose_path):
    if csv.endswith(".csv") or csv.endswith(".CSV"):
        dataframe = pd.read_csv(os.path.join(pose_path, csv), header=None, skiprows=1)
        rolavg = dataframe.rolling(100).mean()
        rolavg = rolavg.dropna()
        rolavg = rolavg.reset_index(drop=True)
        ESdata.append(rolavg.values)
        if rolavg.values.shape[0] < length:
            length = rolavg.values.shape[0]
            print(length)

3181
2170


In [4]:
for idx, array in enumerate(PNESdata):
    PNESdata[idx] = array[:length,::]
    
PNESarray = np.array(PNESdata)
PNESlabel = np.reshape(np.zeros(PNESarray.shape[0]),[PNESarray.shape[0],1])

for idx, array in enumerate(ESdata):
    ESdata[idx] = array[:length,::]
    
ESarray = np.array(ESdata)
ESlabel = np.reshape(np.ones(ESarray.shape[0]),[ESarray.shape[0],1])

X = np.concatenate((PNESarray,ESarray),axis=0)
y = np.concatenate((PNESlabel,ESlabel),axis=0)
y = to_categorical(y)

In [5]:
print(PNESarray.shape)
print(ESarray.shape)
print(PNESlabel.shape)
print(ESlabel.shape)
print(X.shape)
print(y.shape)

(11, 2170, 75)
(16, 2170, 75)
(11, 1)
(16, 1)
(27, 2170, 75)
(27, 2)


In [6]:
outfile = "../Datasets/Pilot_Pose_Video_RolAvg100.npz"
np.savez(outfile, X, y)